## The Composite pattern

The Composite pattern allows complex tree structures to be built from simple
components, often called nodes. A node with children will behave like a container;
a node without children will behave like a single object. A composite object is –
generally – a container object, where the content may be another composite object.
Traditionally, each node in a composite object must be either a leaf node (that cannot
contain other objects) or a composite node. The key is that both composite and leaf
nodes can have the same interface.

![](uml/composite_pattern.png)

### A Composite example

In [93]:
import abc
import argparse
import contextlib
from pathlib import Path
import sys
from typing import Union, Optional, cast


class Node(abc.ABC):
    def __init__(self, name: str) -> None:
        self.name = name
        self.parent: Optional["Folder"] = None
           
    def move(self, new_place: "Folder") -> None:
        previous = self.parent
        new_place.add_child(self)
        if previous:
            del previous.children[self.name]
        
    @abc.abstractmethod
    def copy(self, new_folder: "Folder") -> None: ...
        
    @abc.abstractmethod
    def tree(self, indent: int=0, last: bool=False, outer: bool=False) -> None: ...
        
    @abc.abstractmethod
    def dot(self) -> None: ...
        

In [73]:
class File(Node):
    """
    >>> f = File("name.ex")
    >>> f.tree()
     +-- name.ex
    """
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.name})"
    
    def copy(self, new_folder: "Folder") -> None:
        new_folder.add_child(File(self.name))
    
    def remove(self) -> None:
        if self.parent:
            del self.parent.children[self.name]
    
    def tree(self, indent: int=0, last: bool=False, outer: bool=False) -> None:
        indent_text = "    " if outer else " |   "
        print((indent * indent_text) + " +--", self.name)
        
    def dot(self) -> None:
        print(f"    n{id(self)} [shape=box,label]='{self.name}'")


In [95]:
class Folder(Node):
    """
    >>> fd = {"name1.ex": File("name1.ex"), "name2.ex": File("name2.ex")}
    >>> d = Folder("Folder", fd)
    >>> d.tree()
     +-- Folder
     |    +-- name1.ex
     |    +-- name2.ex
    """
    def __init__(self, name: str, children: Optional[dict[str, "Node"]]=None) -> None:
        super().__init__(name)
        self.children = children or {}
        
    def add_child(self, node: "Node") -> "Node":
        node.parent = self
        return self.children.setdefault(node.name, node)
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.name!r}, {self.children!r})"
    
    def copy(self, new_folder: "Folder") -> None:
        target = cast(Folder, new_folder.add_child(Folder(self.name)))
        
    def remove(self) -> None:
        names = list(self.children)
        for c in names:
            self.children[c].remove()
        if self.parent:
            del self.parent.children[self.name]
        
    def tree(self, indent: int=0, last: bool=False, outer: bool=False) -> None: 
        indent_text = "    " if outer else " |   "
        print((indent * indent_text) + " +--", self.name)
        if self.children:
            *first, final = list(self.children)
            for c in first:
                self.children[c].tree(indent + 1, last=False, outer=outer)
            self.children[final].tree(indent + 1, last=True, outer=outer)
            
    def dot(self) -> None: 
        for c in self.children:
            print(f"    n{id(self)} -> n{id(self.children[c])};")
            self.children[c].dot()
        print(f"    n{id(self)} [label = '{self.name}'];")

    
tree = Folder("Tree")
tree.add_child(Folder("src"))
tree.children["src"].add_child(File("ex1.py"))
tree.add_child(Folder("src"))
tree.children["src"].add_child(File("test1.py"))
tree.tree(outer=True)
print()

test1 = tree.children["src"].children["test1.py"]
print(test1, end="\n\n")

tree.add_child(Folder("tests"))
test1.move(tree.children["tests"])
tree.tree(outer=True)
print()

backup = tree.add_child(Folder("backup"))
test1.copy(backup)
tree.tree()
print()

backup.remove()
tree.tree()

 +-- Tree
     +-- src
         +-- ex1.py
         +-- test1.py

File(test1.py)

 +-- Tree
     +-- src
         +-- ex1.py
     +-- tests
         +-- test1.py

 +-- Tree
 |    +-- src
 |    |    +-- ex1.py
 |    +-- tests
 |    |    +-- test1.py
 |    +-- backup
 |    |    +-- test1.py

 +-- Tree
 |    +-- src
 |    |    +-- ex1.py
 |    +-- tests
 |    |    +-- test1.py
